In [1]:
import pandas as pd
import re

In [2]:
noteText_df = pd.read_csv('../../Data/noteText.csv')
noteText_df.head()

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText
0,D00010F0E618AD,D304C167A23716,2023-09-15,Inpatient Nursing,NaN,Occupational Therapist,OCCUPATIONAL THERAPY CANCELED SESSION NOTE ...
1,D00015E35CE5B7,DD27EC809E2A28,2023-08-25,Inpatient Nursing,UCSF,Resident,History of cardiomyopathy presumed secondary t...
2,D000253FB63C7E,D054778440C77A,2023-09-10,Inpatient Nursing,General Surgery,Physician,Procedure(s) (LRB): KIDNEY TRANSPLANT CADAVER...
3,D0004A42F6443B,D8277269D1C037,2023-08-31,Inpatient Nursing,UCSF,Resident,NEUROSURGERY PROGRESS NOTE Hospital Course...
4,D0005D9BF5CAF1,D5A083F15D8290,2023-09-09,Inpatient Nursing,Nephrology,Physician,TRANSPLANT NEPHROLOGY PROGRESS NOTE Date of...


In [7]:
selected_providers = ['Resident', 'Physician', 'Registered Nurse', 'Nurse Practitioner',
                      'Physician Assistant', 'Pharmacist', 'Licensed Vocational Nurse', 
                      'Medical Student', 'Pharmacy Student', 'Nursing Student', 
                      'Registered Dietitian', 'Dietetic Intern']
selected_noteText_df = noteText_df[(noteText_df['ProviderType'].isin(selected_providers)) | (noteText_df['ProviderType'].isnull())]


In [51]:
# def parse_capital_letters(text):
#     match = re.match(r'^([A-Z]+(?:\s[A-Z]+)*)', text)
#     if match:
#         if len(match.group(0)) > 1:
#             return match.group(0)
#     else:
#         return None
# selected_noteText_df['header'] = selected_noteText_df['NoteText'].apply(parse_capital_letters)

/var/folders/wg/np04l4k14h174_2s7j_7h5kw0000gn/T/ipykernel_3206/1479865787.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_noteText_df['header'] = selected_noteText_df['NoteText'].apply(parse_capital_letters)


In [11]:
# Function to extract surrounding words
def extract_surrounding_text(text):
    match = re.search(r'(\S+\s+){0,6}DISCHARGE(\s+\S+){0,6}', text)
    if match:
        print(match.group(0))
    return match.group(0) if match else None

# Apply the function to the text column
selected_noteText_df['discharge_text'] = selected_noteText_df['NoteText'].apply(extract_surrounding_text)


UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** *****  *****
ORTHOPAEDIC ONCOLOGY: DISCHARGE SUMMARY     Patient ID:  ***** ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
list for detailed instructions):  See above DISCHARGE SUMMARY for medications continued into OSH
up chest pain per usual pathway       DISCHARGE PLANNING  Clinic appointments:   Clinic appointment: Dr
Instructions    ***** ***** ***** ***** AFTER DISCHARGE   ______________________________________________________________________  Attend the post operative appointment
Instructions       UCSF ARTHROPLASTY/JOINT REPLACEMENT SERVICE  PATIENT DISCHARGE INSTRUCTIONS    Name ***** ***** ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient

C:\Users\mevans02\AppData\Local\Temp\ipykernel_10732\1357213087.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_noteText_df['discharge_text'] = selected_noteText_df['NoteText'].apply(extract_surrounding_text)


In [18]:
for text in selected_noteText_df[~selected_noteText_df.discharge_text.isna()].discharge_text.unique():
    print(text)

UCSF MEDICAL CENTER - DISCHARGE SUMMARY - ATTENDING ONLY     Patient Name:
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** *****  *****
ORTHOPAEDIC ONCOLOGY: DISCHARGE SUMMARY     Patient ID:  ***** ***** *****
UCSF MEDICAL CENTER - DISCHARGE SUMMARY     Patient Name: ***** ***** *****
list for detailed instructions):  See above DISCHARGE SUMMARY for medications continued into OSH
up chest pain per usual pathway       DISCHARGE PLANNING  Clinic appointments:   Clinic appointment: Dr
Instructions    ***** ***** ***** ***** AFTER DISCHARGE   ______________________________________________________________________  Attend the post operative appointment
Instructions       UCSF ARTHROPLASTY/JOINT REPLACEMENT SERVICE  PATIENT DISCHARGE INSTRUCTIONS    Name ***** ***** ***** *****
UCSF ARTHROPLASTY/JOINT REPLACEMENT SERVICE  PATIENT DISCHARGE INSTRUCTIONS    Name ***** ***** ***** *****
***** ***** DISCHARGE NOTE       ***** Surgery    Patient: ***** *****
***** ***** DISCHARGE 

In [ ]:
#DISCHARGE NOTE
#DISCHARGE SUMMARY
#DISCHARGE INSTRUCTIONS
#DISCHARGE PLANNING
#DISCHARGE REFERRAL
#CASE MANAGEMENT DISCHARGE
#POST HEART TRANSPLANT DISCHARGE TEACHING

In [53]:
selected_noteText_df['discharge_note'] = selected_noteText_df['header'].apply(lambda x: 1 if x and 'DISCHARGE' in x else 0)

/var/folders/wg/np04l4k14h174_2s7j_7h5kw0000gn/T/ipykernel_3206/1991168358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_noteText_df['discharge_note'] = selected_noteText_df['header'].apply(lambda x: 1 if x and 'DISCHARGE' in x else 0)


In [56]:
selected_noteText_df[selected_noteText_df['discharge_note'] == 1]

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,header,discharge_note
2459,D13A26B78F544E,DEFB3087C5DD6A,2023-09-12,Inpatient Nursing,NaN,Registered Nurse,DISCHARGE Time: 0930 Where: D/C via privat...,DISCHARGE,1
4430,D241902651B4F7,DE4B8024F5C832,2023-09-11,Inpatient Nursing,Cardiology,Nurse Practitioner,CARDIOLOGY DISCHARGE SUMMARY Facility: *...,CARDIOLOGY DISCHARGE SUMMARY,1
7364,D3BE741AA65B7F,D7E9B5A8A51CBC,2023-09-01,Inpatient Nursing,Nurse Practitioner,Nurse Practitioner,CARDIOLOGY DISCHARGE SUMMARY Facility: *...,CARDIOLOGY DISCHARGE SUMMARY,1
11088,D5A22AC0881B61,D3972B7DE02762,2023-09-22,Psychiatry,UCSF,Resident,PSYCHIATRY DISCHARGE SUMMARY Patient Name: ...,PSYCHIATRY DISCHARGE SUMMARY,1
16890,D89F829AF4BF40,DD3EADA5F59CF3,2023-09-09,Inpatient Nursing,Nurse Practitioner,Nurse Practitioner,CARDIOLOGY DISCHARGE SUMMARY Facility: *...,CARDIOLOGY DISCHARGE SUMMARY,1
19910,DA2C12DAAD76BF,DA6D8334B8ABE0,2023-09-11,Inpatient Nursing,NaN,Registered Nurse,DISCHARGE Time:1248 Where: Home AVS Expla...,DISCHARGE,1
20255,DA57DF13480032,D079332799772C,2023-09-15,Inpatient Nursing,Nurse Practitioner,Nurse Practitioner,CARDIOLOGY DISCHARGE SUMMARY Facility: *...,CARDIOLOGY DISCHARGE SUMMARY,1
21418,DAF1916FCCC5FD,D7FF3B8A2845E3,2023-09-28,Psychiatry,UCSF,Resident,PSYCHIATRY DISCHARGE SUMMARY Patient Name: ...,PSYCHIATRY DISCHARGE SUMMARY,1
22355,DB6C5D4735C993,DCD676F7499542,2023-09-20,Inpatient Nursing,UCSF,Resident,DISCHARGE INSTRUCTIONS: Medications: Plea...,DISCHARGE INSTRUCTIONS,1
24118,DC574EBF40EA8D,D856D907D7FFB3,2023-09-05,Psychiatry,UCSF,Resident,PSYCHIATRY DISCHARGE SUMMARY Patient Name: ...,PSYCHIATRY DISCHARGE SUMMARY,1


In [34]:
selected_noteText_df = selected_noteText_df[selected_noteText_df['discharge_note'] == 0].reset_index()
selected_noteText_df

,index,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,header,discharge_note
0,1,D00015E35CE5B7,DD27EC809E2A28,2023-08-25,Inpatient Nursing,UCSF,Resident,History of cardiomyopathy presumed secondary t...,None,0
1,2,D000253FB63C7E,D054778440C77A,2023-09-10,Inpatient Nursing,General Surgery,Physician,Procedure(s) (LRB): KIDNEY TRANSPLANT CADAVER...,None,0
2,3,D0004A42F6443B,D8277269D1C037,2023-08-31,Inpatient Nursing,UCSF,Resident,NEUROSURGERY PROGRESS NOTE Hospital Course...,NEUROSURGERY PROGRESS NOTE,0
3,4,D0005D9BF5CAF1,D5A083F15D8290,2023-09-09,Inpatient Nursing,Nephrology,Physician,TRANSPLANT NEPHROLOGY PROGRESS NOTE Date of...,TRANSPLANT NEPHROLOGY PROGRESS NOTE,0
4,5,D00061A8A25CE6,D8017C77BA15FA,2023-07-09,Inpatient Nursing,Nephrology,Physician,NEPHROLOGY FOLLOW UP CONSULT NOTE Events ...,NEPHROLOGY FOLLOW UP CONSULT NOTE,0
...,...,...,...,...,...,...,...,...,...,...
25410,31210,DFFF59E5F2996F,D8E534C1220218,2023-09-06,Inpatient Nursing,NaN,Registered Nurse,Wound Care Reason for consult: stoma marki...,None,0
25411,31212,DFFF8CBF734480,D70969A027A062,2023-06-23,Inpatient Nursing,NaN,Registered Nurse,Mechanical Circulatory Support Indicatio...,None,0
25412,31213,DFFF9A013C9A3D,DC5430B22637DD,2023-08-30,Inpatient Nursing,NaN,Pharmacist,ANTIMICROBIAL STEWARDSHIP CONSULT This p...,None,0
25413,31214,DFFFB1BAD49145,D70969A027A062,2023-07-22,Inpatient Nursing,UCSF,Resident,Cardiac Surgery Lung Transplant Progress Note ...,None,0


In [38]:
selected_noteText_df.iloc[2]['NoteText']

'NEUROSURGERY PROGRESS NOTE     Hospital Course  August 22  L5-S1 *****     August 29 T4-Pelvis PSF L3 PSO T12-L2 lami     Subjective  resting    Vitals  Temp:  [36.7 °C (98 °F)-36.9 °C (98.4 °F)] 36.7 °C (98.1 °F)  Pulse:  [73-103] 76  *****Resp:  [13-18] 15  BP: (127-171)/(54-95) 154/95  Arterial Line BP (mmHg) : (91)/(63) 91/63  SpO2:  [92 %-97 %] 95 %    Input / Output  I/O last 2 completed shifts plus current shift:  In: 1321.8 [P.O.:420; I.V.:11.8; IV Piggyback:250]  Out: 4738 [Urine:3803; Drains/*****:*****]    Physical Exam  See below    Labs  Lab Results   Component Value Date/Time    WBC 15.3 (H) 08/30/2023 11:21 PM    HCT 25.6 (L) 08/30/2023 11:21 PM    PLT 141 08/30/2023 11:21 PM    NA 134 (L) 08/30/2023 10:05 AM    NAWB 135 (L) 08/28/2023 02:22 PM    K 4.8 08/30/2023 10:05 AM    KSB 4.5 08/28/2023 02:22 PM    CL 98 (L) 08/30/2023 10:05 AM    CLWB 100 08/28/2023 02:***** *****    ***** 29 08/30/2023 10:05 AM    HCO3 28 08/28/2023 02:22 PM    PT 14.2 08/30/2023 02:51 AM    P

In [57]:
def split_by_multiple_spaces(text):
    return re.split(r'\s{3,}', text)
split_by_multiple_spaces(selected_noteText_df.iloc[3]['NoteText'])

['TRANSPLANT NEPHROLOGY PROGRESS NOTE',
 'Date of service: 09/09/2023',
 'Name of the patient: ***** *****  Date of birth: 02/15/2003  MRN: *****',
 'Indication for kidney biopsy: Cause biopsy for elevated SCr',
 "Assessment:  I personally reviewed the patient's medications blood pressure and laboratory data.",
 'I have reviewed the indication for the biopsy.',
 'Vitals:',
 '09/09/23 0808',
 'BP: (!) 122/89',
 'BP Location: Right upper arm',
 'Patient Position: Lying',
 'Pulse: 78',
 'Resp: 16',
 'Temp: 36.8 °C (98.3 °F)',
 'TempSrc: Oral',
 'SpO2: 100%',
 'Recent Labs',
 '09/09/23  0841 09/07/23  1125',
 'WBC 7.5 7.8',
 'HGB 9.6***** 10.8*****',
 'HCT 28.2***** 33.3*****',
 'PLT ***** *****',
 '***** ***** 129*****',
 '***** 5.0 5.9*****',
 '***** ***** *****',
 'CO2 18***** 19*****',
 'BUN 42***** 49*****',
 'CREAT 2.69***** 2.54*****',
 'GLU 94 89',
 'PT 15.3*****  --',
 'INR 1.2  --',
 'PTT 36.0*****  --',
 'Bleeding Risk Assessment:  The patient is not taking any antiplatelet agen